In [ ]:
# Libraries

import datetime
import json
import numpy
import os
import platform
import shutil
import sys
import torch

In [ ]:
# Root directory

ROOT = os.getcwd()

while not ROOT.endswith('upolanc-thesis') :
	ROOT = os.path.abspath(os.path.join(ROOT, os.pardir))

	if len(ROOT) < len('upolanc-thesis') :
		if   platform.system() == 'Linux' :
			ROOT = '/d/hpc/home/up4472/workspace/upolanc-thesis'
		elif platform.system() == 'Windows' :
			ROOT = 'C:\\Developer\\Workspace\\PyCharm\\Projects\\upolanc-thesis'
		else :
			raise ValueError()

		print(f'Warning : could not find correct directory, using default : {ROOT}')
		break

if ROOT not in sys.path :
	sys.path.append(ROOT)

os.chdir(ROOT)

In [ ]:
# Tune

from ray.air                  import CheckpointConfig
from ray.air                  import FailureConfig
from ray.air                  import RunConfig
from ray.tune                 import JupyterNotebookReporter
from ray.tune                 import TuneConfig
from ray.tune                 import Tuner
from ray.tune.logger          import CSVLoggerCallback
from ray.tune.schedulers      import ASHAScheduler
from ray.tune.stopper         import TimeoutStopper
from ray.tune.search          import ConcurrencyLimiter
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.hyperopt import HyperOptSearch

from ray import tune

In [ ]:
# Code

from source.python.cnn import core    as cnn_core
from source.python.cnn import dataset as cnn_dataset
from source.python.cnn import raytune as cnn_raytune
from source.python.io  import loader  as data_loader

# 1. Setup

In [ ]:
# Setup model to tune

MODEL_NAME = 'zrimec2020'
MODEL_TYPE = 'r'

In [ ]:
# Setup some directory paths.

CWD = ROOT
OUT = os.path.join(CWD, 'output')
RES = os.path.join(CWD, 'resources')

OUT_DATA  = os.path.join(OUT,      'nbp06-tuner', MODEL_NAME)
OUT_PLOT  = os.path.join(OUT_DATA, 'plot')
RES_TUNER = os.path.join(RES,      'tuner')
RES_NBP04 = os.path.join(OUT,      'nbp04-feature')
RES_NBP05 = os.path.join(OUT,      'nbp05-target')

shutil.rmtree(OUT_DATA, ignore_errors = True)

os.makedirs(OUT_DATA, exist_ok = True)
os.makedirs(OUT_PLOT, exist_ok = True)

print(f'     Root Directory : {CWD}')
print(f'   Output Directory : {OUT_DATA}')
print(f'   Output Directory : {OUT_PLOT}')
print(f' Resource Directory : {RES_TUNER}')
print(f' Resource Directory : {RES_NBP04}')
print(f' Resource Directory : {RES_NBP05}')

In [ ]:
# System and device

DEVICE = cnn_core.get_device(only_cpu = False)
SYSTEM = cnn_core.get_system_info()

for key, value in SYSTEM.items() :
	print('{:25s} : {}'.format(key, value))

In [ ]:
# Load the input transcript features

feature_sequence = data_loader.load_fasta(
	filename  = os.path.join(RES_NBP04, 'features-bp2150.fasta'),
	to_string = True
)

feature_base = data_loader.load_npz(
	filename = os.path.join(RES_NBP04, 'features-base.npz')
)

# 2. CNN Regression

## 2.1 Config

In [ ]:
# Define basic data parameters

CONFIG = {
	'core/random'           : None,
	'core/device'           : DEVICE,
	'core/rootdir'          : ROOT,
	'dataset/filepath'      : os.path.join(OUT_DATA, 'dataset.pkl'),
	'dataset/expanddim'     : None,
	'dataset/split/valid'   : 0.2,
	'dataset/split/test'    : 0.2,
	'tuner/trials'          : 5,
	'tuner/max_epochs'      : 50,
	'tuner/min_epochs'      : 5,
	'tuner/max_concurrent'  : 1,
	'tuner/notebook'        : True,
	'tuner/checkpoint'      : False,
	'tuner/stopper'         : True,
	'tuner/stopper/days'    : 3,
	'tuner/stopper/hours'   : 18,
	'tuner/stopper/minutes' : 0,
	'model/name'            : MODEL_NAME,
	'model/type'            : MODEL_NAME + MODEL_TYPE,
	'model/epochs'          : 15,
	'model/input/channels'  : 1,
	'model/input/height'    : 4,
	'model/input/width'     : 2150,
	'model/input/features'  : None,
	'model/output/target'   : 'tissue',
	'model/output/type'     : 'mean',
	'model/output/filter'   : None,
	'model/output/size'     : None
}

CONFIG['core/random'] = cnn_core.lock_random(
	seed     = CONFIG['core/random'],
	generate = True
)

if CONFIG['model/name'].startswith('washburn2019') :
	CONFIG['dataset/expanddim'] = 0

In [ ]:
# Prints

print('Model Name  : {}'.format(CONFIG['model/name']))
print('Model Type  : {}'.format(CONFIG['model/type']))
print('Random Seed : {}'.format(CONFIG['core/random']))

In [ ]:
# Load features

dataframe, target_value, target_order = data_loader.load_feature_targets(
	group    = '{}-{}'.format(CONFIG['model/output/target'], CONFIG['model/output/type']),
	explode  = False,
	filters  = {
		'tissue'       : None,
		'group'        : None,
		'age'          : None,
		'perturbation' : None
	},
	directory = RES_NBP05,
	filename  = 'mapping-{}-grouped.pkl'.format(CONFIG['model/output/type'])
)

if 'Feature' in dataframe.columns :
	feature_base = {
		key : numpy.concatenate((feature_base[key.split('?')[-1]], value))
		for key, value in dataframe['Feature'].to_dict().items()
	}

CONFIG['model/output/size']    = len(target_order)
CONFIG['model/input/features'] = len(list(feature_base.values())[0])

dataframe

In [ ]:
# Create dataset

dataset = cnn_dataset.to_dataset(
	sequences   = feature_sequence,
	features    = feature_base,
	targets     = target_value,
	expand_dims = CONFIG['dataset/expanddim']
)

In [ ]:
# Save dataset

torch.save(dataset, CONFIG['dataset/filepath'])

In [ ]:
# Assert all is set

if CONFIG['dataset/filepath']     is None : raise AssertionError('dataset/filepath')
if CONFIG['model/input/features'] is None : raise AssertionError('model/input/features')
if CONFIG['model/output/size']    is None : raise AssertionError('model/output/size')

print('Dataset Filepath : {}'.format(CONFIG['dataset/filepath']))
print('Input Features   : {}'.format(CONFIG['model/input/features']))
print('Output Size      : {}'.format(CONFIG['model/output/size']))

## 2.2 Search Space

In [ ]:
# Define optimizer parameters

optimizer_name     = tune.choice(['adam', 'sgd'])
optimizer_lr       = tune.loguniform(1e-5, 1e-2)
optimizer_decay    = tune.uniform(0.00, 0.10)
optimizer_momentum = tune.uniform(0.50, 0.99)

# TODO - replace optimizer_momentum
optimizer_adam_beta1   = tune.uniform(0.500, 0.990)
optimizer_adam_beta2   = tune.uniform(0.900, 0.999)
optimizer_sgd_momentum = tune.uniform(0.500, 0.990)

In [ ]:
# Define scheduler parameters

scheduler_name             = tune.choice(['constant', 'step', 'plateau', 'linear', 'exponential'])
scheduler_plateau_factor   = tune.uniform(0.1, 0.5)
scheduler_plateau_patience = tune.randint(1, 25)
scheduler_step_factor      = tune.uniform(0.1, 0.5)
scheduler_step_patience    = tune.randint(1, 25)
scheduler_linear_factor    = tune.uniform(1e-05, 1e-01)
scheduler_exp_factor       = tune.uniform(0.950, 0.999)

In [ ]:
# Define model parameters

model_dropout  = tune.quniform(0.0, 0.5, 0.01)

conv1_kernels  = [5, 11, 21, 31, 41]
conv1_filters  = [32, 64, 128, 256]

convx_kernels  = [5, 11, 21, 31, 41]
convx_padding  = ['same', 'none']
convx_filters  = [32, 64, 128, 256]
convx_dilation = [1, 2, 4]

poolx_kernels  = [3, 5, 9]
poolx_padding  = ['same', 'none']

fcx_features = [64, 128, 256, 512]

if CONFIG['model/name'].startswith('washburn2019') :
	conv1_kernels = [(4, x) for x in conv1_kernels]
	convx_kernels = [(1, x) for x in convx_kernels]
	poolx_kernels = [(1, x) for x in poolx_kernels]

In [ ]:
# Define search space (tested with zrimec2020 and washburn2019)

param_space = dict()

# Dataset
param_space['dataset/batch_size'] = tune.choice([32, 64, 128])

# Optimizer
param_space['optimizer/name']     = optimizer_name
param_space['optimizer/lr']       = optimizer_lr
param_space['optimizer/decay']    = optimizer_decay
param_space['optimizer/momentum'] = optimizer_momentum

# Criterion
param_space['criterion/name']      = 'mse'
param_space['criterion/reduction'] = 'mean'

# Scheduler
param_space['scheduler/name']               = scheduler_name
param_space['scheduler/plateau/factor']     = scheduler_plateau_factor
param_space['scheduler/plateau/patience']   = scheduler_plateau_patience
param_space['scheduler/linear/factor']      = scheduler_linear_factor
param_space['scheduler/step/factor']        = scheduler_step_factor
param_space['scheduler/step/patience']      = scheduler_step_patience
param_space['scheduler/exponential/factor'] = scheduler_exp_factor

# Model
param_space['model/dropout'] = model_dropout

# Convolution
param_space['model/conv1/filters']  = tune.choice(conv1_filters)
param_space['model/conv1/kernel']   = tune.choice(conv1_kernels)
param_space['model/conv1/padding']  = 'none'
param_space['model/conv1/dilation'] = 1
param_space['model/conv2/filters']  = tune.choice(convx_filters)
param_space['model/conv2/kernel']   = tune.choice(convx_kernels)
param_space['model/conv2/padding']  = tune.choice(convx_padding)
param_space['model/conv2/dilation'] = 1
param_space['model/conv3/filters']  = tune.choice(convx_filters)
param_space['model/conv3/kernel']   = tune.choice(convx_kernels)
param_space['model/conv3/padding']  = tune.choice(convx_padding)
param_space['model/conv3/dilation'] = 1

if CONFIG['model/name'].startswith('washburn2019') :
	param_space['model/conv4/filters']  = tune.choice(convx_filters)
	param_space['model/conv4/kernel']   = tune.choice(convx_kernels)
	param_space['model/conv4/padding']  = tune.choice(convx_padding)
	param_space['model/conv4/dilation'] = 1
	param_space['model/conv5/filters']  = tune.choice(convx_filters)
	param_space['model/conv5/kernel']   = tune.choice(convx_kernels)
	param_space['model/conv5/padding']  = tune.choice(convx_padding)
	param_space['model/conv5/dilation'] = 1
	param_space['model/conv6/filters']  = tune.choice(convx_filters)
	param_space['model/conv6/kernel']   = tune.choice(convx_kernels)
	param_space['model/conv6/padding']  = tune.choice(convx_padding)
	param_space['model/conv6/dilation'] = 1

# Pooling
param_space['model/maxpool1/kernel']  = tune.choice(poolx_kernels)
param_space['model/maxpool1/padding'] = 'same'
param_space['model/maxpool2/kernel']  = tune.choice(poolx_kernels)
param_space['model/maxpool2/padding'] = 'same'
param_space['model/maxpool3/kernel']  = tune.choice(poolx_kernels)
param_space['model/maxpool3/padding'] = 'same'

# Linear
param_space['model/fc1/features'] = tune.choice(fcx_features)
param_space['model/fc2/features'] = tune.choice(fcx_features)

In [ ]:
# Define starting parameters for algorithm (some decent starting point from previous runs)

params_initial = None

if CONFIG['model/type'].startswith('zrimec2020r') :
	filename = os.path.join(RES_TUNER, 'zrimec2020.json')

	if os.path.exists(filename) :
		params_initial = data_loader.load_json(filename = filename)

# DOES NOT WORK WITH (X, Y) KERNELS AND PADDING
if CONFIG['model/type'].startswith('washburn2019r') :
	filename = os.path.join(RES_TUNER, 'washburn2019.json')

	if os.path.exists(filename) :
		params_initial = data_loader.load_json(filename = filename)

## 2.3 Tuning

In [ ]:
# Create a tune method

tune_method = tune.with_resources(
	lambda x : cnn_raytune.regression_tune(
		tune_config = x,
		core_config = CONFIG
	),
	{
		'cpu' : max(1, SYSTEM['platform/cpu/count'] // 2),
		'gpu' : max(0, SYSTEM['platform/gpu/count'])
	}
)

In [ ]:
# Create tune config

tune_searcher = HyperOptSearch(
	points_to_evaluate = params_initial,
	metric             = 'valid_loss',
	mode               = 'min'
)

tune_searcher = ConcurrencyLimiter(
	tune_searcher,
	max_concurrent = CONFIG['tuner/max_concurrent']
)

tune_scheduler = ASHAScheduler(
	time_attr        = 'training_iteration',
	max_t            = CONFIG['tuner/max_epochs'],
	grace_period     = CONFIG['tuner/min_epochs'],
	reduction_factor = 4,
	brackets         = 1
)

tune_config = TuneConfig(
	metric      = 'valid_loss',
	mode        = 'min',
	num_samples = CONFIG['tuner/trials'],
	search_alg  = tune_searcher,
	scheduler   = tune_scheduler,
	trial_name_creator    = lambda x : str(x.trial_id),
	trial_dirname_creator = lambda x : str(x.trial_id)
)

In [ ]:
# Create tune run config

VERBOSE_SILENT = 0
VERBOSE_STATUS = 1
VERBOSE_BRIEF  = 2
VERBOSE_DETAIL = 3

reporter = None
stopper  = None

if CONFIG['tuner/notebook'] :
	reporter = JupyterNotebookReporter(
		max_column_length = 32,
		max_progress_rows = 20,
		parameter_columns = ['dataset/batch_size', 'optimizer/name', 'optimizer/lr', 'scheduler/name', 'model/dropout'],
		metric_columns    = ['valid_loss', 'valid_r2', 'train_loss']
	)

failure = FailureConfig(
	max_failures = 0
)

checkpoint = CheckpointConfig(
	num_to_keep = None
)

if CONFIG['tuner/stopper'] :
	stopper = TimeoutStopper(
		timeout = datetime.timedelta(
			days    = CONFIG['tuner/stopper/days'],
			hours   = CONFIG['tuner/stopper/hours'],
			minutes = CONFIG['tuner/stopper/minutes']
		)
	)

run_config = RunConfig(
	name              = 'raytune',
	local_dir         = OUT_DATA,
	callbacks         = None,
	log_to_file       = True,
	stop              = stopper,
	verbose           = VERBOSE_STATUS,
	failure_config    = failure,
	checkpoint_config = checkpoint,
	progress_reporter = reporter
)

In [ ]:
# Create tune and fit

tuner = Tuner(
	trainable   = tune_method,
	tune_config = tune_config,
	run_config  = run_config,
	param_space = param_space
)

report = tuner.fit()

## 2.4 Reports

In [ ]:
# Check if there have been errors

if report.errors :
	print('At least one of the trials has failed.')
else :
	print('No errors.')

In [ ]:
# Print possible columns

for index, column in enumerate(report.get_dataframe().columns, start = 1) :
	print('{:<40s}'.format(column), end = ' ')

	if index % 3 == 0 :
		print()

In [ ]:
# Print the results as dataframe

dataframe = report.get_dataframe()

print('Shortest training time : {:8,.1f} minutes'.format(dataframe['time_total_s'].min()  / 60.0))
print(' Longest training time : {:8,.1f} minutes'.format(dataframe['time_total_s'].max()  / 60.0))
print('   Total training time : {:8,.1f} minutes'.format(dataframe['time_total_s'].sum()  / 60.0))
print('    Mean training time : {:8,.1f} minutes'.format(dataframe['time_total_s'].mean() / 60.0))
print()

trials = dataframe[['trial_id', 'logdir', 'time_total_s', 'training_iteration', 'valid_loss', 'valid_r2', 'valid_mae', 'train_loss']]
trials = trials.sort_values('valid_loss', ascending = True)
trials

### 2.4.1 VIsualization

In [ ]:
# Plot trials valid loss

cnn_raytune.plot_trials_loss(
	dataframe  = trials,
	max_trials = 10,
	alpha      = 0.9,
	filename   = os.path.join(OUT_PLOT, CONFIG['model/type'] + '-trials')
)

In [ ]:
# Plot trials valid r2

cnn_raytune.plot_trials_r2(
	dataframe  = trials,
	max_trials = 10,
	alpha      = 0.9,
	filename   = os.path.join(OUT_PLOT, CONFIG['model/type'] + '-trials')
)

### 2.4.2 Best

In [ ]:
# Display best trial

best = report.get_best_result()

print('Best metric : loss = {: 8.5f} r2 = {: 8.5f}'.format(best.metrics['valid_loss'], best.metrics['valid_r2']))
print('Best config : ')

for key, val in best.config.items() :
	if isinstance(key, str) : key = f'"{key}"'
	if isinstance(val, str) : val = f'"{val}"'

	print('{:30s} : {}'.format(key, val))

In [ ]:
# Display per iteration

progress = data_loader.load_csv(
	filename = os.path.join(best.log_dir, 'progress.csv')
)

progress = progress[['trial_id', 'experiment_id', 'time_total_s', 'training_iteration', 'lr', 'valid_loss', 'valid_r2', 'valid_mae', 'train_loss']]
progress

In [ ]:
# Plot best trial valid loss

cnn_raytune.plot_trial_loss(
	dataframe = progress,
	filename  = os.path.join(OUT_PLOT, CONFIG['model/type'] + '-best')
)

In [ ]:
# Plot best trial valid r2

cnn_raytune.plot_trial_r2(
	dataframe = progress,
	filename  = os.path.join(OUT_PLOT, CONFIG['model/type'] + '-best')
)

In [ ]:
# Plot trials learning rate

cnn_raytune.plot_trial_lr(
	dataframe = progress,
	filename  = os.path.join(OUT_PLOT, CONFIG['model/type'] + '-best')
)

### 2.4.3 Worst

In [ ]:
# Display worst trial

worst = report.get_best_result(metric = 'valid_loss', mode = 'max')

print('Worst metric : loss = {: 8.5f} r2 = {: 8.5f}'.format(worst.metrics['valid_loss'], worst.metrics['valid_r2']))
print('Worst config : ')

for key, val in best.config.items() :
	if isinstance(key, str) : key = f'"{key}"'
	if isinstance(val, str) : val = f'"{val}"'

	print('{:30s} : {}'.format(key, val))

In [ ]:
# Display per iteration

progress = data_loader.load_csv(
	filename = os.path.join(worst.log_dir, 'progress.csv')
)

progress = progress[['trial_id', 'experiment_id', 'time_total_s', 'training_iteration', 'lr', 'valid_loss', 'valid_r2', 'valid_mae', 'train_loss']]
progress

## 2.5 Save

In [ ]:
# Save the tuning results

report.get_dataframe().to_csv(
	os.path.join(OUT_DATA, CONFIG['model/type'] + '-report.csv')
)